# Generating Contact Networks

In [33]:
import pandas as pd
import random

In [34]:
def pick(probabilities):
    val = random.random()
    for i, probability in enumerate(probabilities):
        if val <= probability:
            return i

In [35]:
class Person:
    count = 0
    people = []

    def __init__(self, family, age=None, apartment=-1, home=-1):
        self.id = Person.count
        self.age = random.randint(20, 40) if age == None else random.randint(age[0], age[1])
        self.apartment = apartment
        self.home = home
        self.family = family
        self.office = -1
        self.school = -1

        Person.count = Person.count + 1

    def __repr__(self):
        return str({
            'id': self.id,
            'age': self.age,
            'apartment': self.apartment,
            'home': self.home,
            'family': self.family,
            'office': self.office,
            'school': self.school
        })

In [36]:
class Family:
    # assuming a city population
    # imagine families to be based around 1 or 2 adults of age [30, 60]
    # having chance c of having children 1, 2 or 3 children of age [1, m - [25-30]]
    # where m in the age of the younger adult
    # and having chance p of living with 0, 1 or 2 of their parents
    # whose age would be in range [M + [20-25], 85]
    count = 0

    a = [0, 0.65, 1]
    c = [0.2, 0.45, 0.9, 1]
    p = [0.5, 0.65, 1]

    def __init__(self, n=None, apartment=-1, home=-1):
        self.members = []
        self.id = Family.count
        Family.count = Family.count + 1

        if n is not None:
            for _ in range(n):
                person = Person(family=self.id, apartment=apartment, home=home)
                Person.people.append(person)
                self.members.append(person.id)
    
        adults = pick(Family.a)
        children = pick(Family.c)
        parents = pick(Family.p)

        m = 100
        M = -1

        for _ in range(adults):
            person = Person(age=(30, 50), family=self.id, apartment=apartment, home=home)
            Person.people.append(person)
            m = min(m, person.age)
            M = max(M, person.age)
            self.members.append(person.id)

        for _ in range(children):
            person = Person(age=(1, m - random.randint(25, 28)), apartment=apartment, home=home, family=self.id)
            Person.people.append(person)
            self.members.append(person.id)

        for _ in range(parents):
            person = Person(age=(M + 30, M + 35), apartment=apartment, home=home, family=self.id)
            Person.people.append(person)
            self.members.append(person.id)

    def __repr__(self):
        return str(self.id) + ":" + str(self.members)

In [37]:
class Home:
    count = 0

    def __init__(self):
        self.id = Home.count
        self.family = Family(home=self.id)
        Home.count = Home.count + 1

    def __repr__(self):
        return str(self.id) + ":" + str(self.family)

In [38]:
class Apartment:
    size = 30
    count = 0
    
    def __init__(self):
        self.families = [Family(apartment=Apartment.count) for _ in range(Apartment.size)]
        Apartment.count = Apartment.count + 1

    def __repr__(self):
        return str(self.families)

In [39]:
class Office:
    count = 0

    def __init__(self):
        self.id = Office.count
        Office.count = Office.count + 1
        self.employees = set()

    def hire(self, id):
        Person.people[id].office = self.id
        self.employees.add(id)

    def __repr__(self):
        return str(self.id) + ":" + str(self.employees)

In [40]:
class School:
    count = 0

    def __init__(self):
        self.id = School.count
        School.count = School.count + 1
        self.students = set()

    def admit(self, id):
        Person.people[id].school = self.id
        self.students.add(id)

    def __repr__(self):
        return str(self.id) + ":" + str(self.students)

In [41]:
class Society:
    
    def __init__(
        self,
        apartments=3,
        homes=15,
    ):
        self.n_apartments = apartments
        self.apartments = [Apartment() for _ in range(apartments)]

        self.n_homes = homes
        self.homes = [Home() for _ in range(homes)]

        self.n_offices = Person.count // 50 + 1
        self.offices = [Office() for _ in range(self.n_offices)]
        
        self.pre = School()
        self.primary = School()
        self.secondary = School()
        self.college = School()
        self.uni = School()
        
        for person in Person.people:
            if person.age > 65:
                continue
            if person.age >= 25:
                if random.random() <= 0.6:
                    self.offices[random.randint(1, self.n_offices) - 1].hire(person.id)
            elif person.age >= 18:
                if random.random() <= 0.7:
                    self.uni.admit(person.id)
            elif person.age >= 16:
                if random.random() <= 0.75:
                    self.college.admit(person.id)
            elif person.age >= 11:
                if random.random() <= 0.9:
                    self.secondary.admit(person.id)
            elif person.age >= 5:
                if random.random() >= 0.95:
                    self.primary.admit(person.id)
            else:
                if random.random() >= 0.99:
                    self.pre.admit(person.id)
        

        
        

In [42]:
society = Society()

In [43]:
id = [person.id for person in Person.people]
age = [person.age for person in Person.people]
apartment = [person.apartment for person in Person.people]
home = [person.home for person in Person.people]
family = [person.family for person in Person.people]
office = [person.office for person in Person.people]
school = [person.school for person in Person.people]

In [45]:
data = {
    'id': id,
    'age': age,
    'apartment': apartment,
    'home': home,
    'family': family,
    'office': office,
    'school': school
}

meta = pd.DataFrame(data)

In [46]:
meta.head()

,id,age,apartment,home,family,office,school
0,0,34,0,-1,0,-1,-1
1,1,36,0,-1,0,0,-1
2,2,6,0,-1,0,-1,-1
3,3,3,0,-1,0,-1,-1
4,4,31,0,-1,1,5,-1


In [47]:
meta.describe()

,id,age,apartment,home,family,office,school
count,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000
mean,194.500000,34.641026,0.692308,-0.051282,50.166667,0.138462,-0.697436
std,112.727548,27.256647,1.030483,2.866783,30.670995,2.342479,0.983537
min,0.000000,1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000
25%,97.250000,7.000000,0.000000,-1.000000,22.250000,-1.000000,-1.000000
50%,194.500000,35.500000,1.000000,-1.000000,50.500000,-1.000000,-1.000000
75%,291.750000,50.000000,2.000000,-1.000000,77.000000,-1.000000,-1.000000
max,389.000000,85.000000,2.000000,14.000000,104.000000,7.000000,4.000000


In [48]:
meta.to_csv('metadata.csv', header=True, index=False, sep='\t')